In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [17]:
random.seed(1344)
g = zx.generate.cliffordT(qubits=8, depth=500, p_t=0.3)
c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 8 qubits with 500 gates.
        155 is the T-count
        345 Cliffords among which 
        114 2-qubit gates (114 CNOT, 0 other) and
        0 Hadamard gates.


In [18]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
zx.Circuit.from_graph(g).split_phase_gates().stats()

'Circuit  on 8 qubits with 349 gates.\n        69 is the T-count\n        280 Cliffords among which \n        114 2-qubit gates (107 CNOT, 7 other) and\n        73 Hadamard gates.'

In [19]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
zx.extract_circuit(g_full.copy()).stats()

'Circuit  on 8 qubits with 381 gates.\n        69 is the T-count\n        312 Cliffords among which \n        214 2-qubit gates (34 CNOT, 180 other) and\n        93 Hadamard gates.'

In [24]:
g_greedy = g_tele.copy()
zx.simplify.greedy_simp(g_greedy)
zx.extract_circuit(g_greedy.copy()).stats()

spider_simp: 81. 47. 22. 13. 5. 1.  6 iterations
spider_simp: 5. 1.  2 iterations
id_simp: 6. 3.  2 iterations


'Circuit  on 8 qubits with 368 gates.\n        69 is the T-count\n        299 Cliffords among which \n        106 2-qubit gates (2 CNOT, 104 other) and\n        153 Hadamard gates.'

In [20]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
zx.extract_circuit(g_nu.copy()).stats()

spider_simp: 81. 47. 22. 13. 5. 1.  6 iterations
spider_simp: 5. 1.  2 iterations
id_simp: 6. 3.  2 iterations


'Circuit  on 8 qubits with 369 gates.\n        69 is the T-count\n        300 Cliffords among which \n        105 2-qubit gates (2 CNOT, 103 other) and\n        155 Hadamard gates.'